In [1]:
import syft as sy
from syft.types.datetime import DateTime
from syft.service.metadata.node_metadata import NodeMetadata, NodeMetadataV1
from syft.service.metadata.node_metadata import NodeMetadataJSON

In [2]:
from syft.protocol.data_protocol import DataProtocol

In [3]:
from syft.service.service import BaseConfig

In [4]:
from syft.store.dict_document_store import DictStoreConfig

In [5]:
sy.stage_protocol_changes()

SyftSuccess: 110 Protocol Updates Staged to dev

In [6]:
sy.stage_protocol_changes()

SyftSuccess: 0 Protocol Updates Staged to dev

In [7]:
sy.bump_protocol_version()

SyftSuccess: Protocol Updated to 1

In [ ]:
x = sy.get_data_protocol()

In [ ]:
x.supported_protocols

In [ ]:
x.state

In [ ]:
x.state["PartialSyftObject"]

In [ ]:
import re

In [ ]:
def natural_key(key: int | str) -> list[int]:
    """Define key for natural ordering of strings."""
    if isinstance(key, int):
        key = str(key)
    return [int(s) if s.isdigit() else s for s in re.split("(\d+)", key)]

In [ ]:
x = ["dev"]

In [ ]:
sorted(x, key=natural_key)

In [ ]:
raise Exception

In [ ]:
import re


In [ ]:
def natural_key(key: int | str) -> list[int]:
    """Define key for natural ordering of strings."""
    if isinstance(key, int):
        key = str(key)
    return [int(s) if s.isdigit() else s for s in re.split("(\d+)", key)]

In [ ]:
a = ["1", "dev", "2", "11"]

In [ ]:
b = sorted(a, key=natural_key)

In [ ]:
b

In [ ]:
DataProtocol._calculate_object_hash(BaseConfig)

In [ ]:
from syft.serde.recursive import TYPE_BANK

In [ ]:
g = {"1":"a", "22":"b", "11":"c"}

In [ ]:
max(g)

In [ ]:
range(2, 23)

In [ ]:
(
nonrecursive,
serialize,
deserialize,
attribute_list,
exclude_attrs_list,
serde_overrides,
hash_exclude_attrs,
cls,
attribute_types,
version,
) = TYPE_BANK["syft.service.service.BaseConfig"]

In [ ]:
cls

In [ ]:
cls == BaseConfig

In [ ]:
DataProtocol._calculate_object_hash(cls)

In [ ]:

# Exception: BaseConfig <class 'syft.service.service.ServiceConfig'> version 1 hash has changed.

# 332964663526a4d6ddc64bc9d77aa9324000bb721520a5465229235523fe8c7d not in 

# dict_values([{'version': 1, 'hash': 'f35db90c1c3bdd993b7b1e4f28327a8f6b1d0e43115b044167ca28d5c740178a', 'action': 'add'}]). You probably need to bump the version number.


In [ ]:
raise Exception

In [ ]:
x = {'PartialSyftObject': {1: 'fa2770d76f3dd904e2c1c4a78c9ba10e5ac33b4dd7a4d61faa45a22078e94aa8'}, 'NodeMetadataUpdate': {1: 'b73adf379a179f6ef9a6d5ee9477e69697a47ad25579bab46cc1fc44ec35ba04'}, 'NodeMetadata': {2: '3986c7d8b3cf9355a1fbdd99dfe3d872fc464420b91a668ea3288ee4481bab6d'}, 'LinkedObject': {1: '8a117f8bf0282b8cf525de893404dbd88cc460a5a823d83006c2e82d88846f8d'}, 'BaseConfig': {1: '32b71d31ecba9a29c82fc4ca0e6568ac5c97204501ffdfef079c93ffbaf6b278'}, 'APIEndpoint': {1: 'c88e4405839e87fdfe90f86877ef2addd7be7281d36b7891636129fc8b3c1e8c'}, 'LibEndpoint': {1: '214b81bda5fd00e7734227a4c75981d807f3741039be5b4fb007076cf2e638cc'}, 'SignedSyftAPICall': {1: '2be9b74663354b4edeef3bc75dc67dc35bf24890c8a86a53a97957d470af06b2'}, 'SyftAPICall': {1: 'fa9520d29d9df56fb9d5d2080aecfc3be14c49e7267a3e5b9fd05ad1b0828e11'}, 'SyftAPIData': {1: '16933cd10a2c45ad44826480e88e06ffbb7a7762c058c7e84da94ddc70478e7c'}, 'SyftAPI': {1: '142a9bb5a4a63d5d072fbfbdbb48ec9535f1ae51e40f9d4f59760fb807c9a189'}, 'User': {1: '21cb3659dc4ddd0b4d58c677f46feecac9f682ea36660f42845067b29b5ad8e7'}, 'UserUpdate': {1: 'f12c19dd38330f98fb2d9e0bf47e8bdcad3f6e1c085d2994d80224cf4b905984'}, 'UserCreate': {1: 'a9d7a52aaa7dcf622e317e899e1ded3023a94b86773ca16cd7d6a334fcffbe8b'}, 'UserSearch': {1: 'e697bf5b287cf29560c94c5851d8fb6ac74d2ce5c6200539a11a257bc150c75b'}, 'UserView': {1: 'fd624963af09c0e3471dfc49b2f09fafdd7521c8af804198015cc5455d7b56bc'}, 'UserViewPage': {1: 'c856f13ddc9a405b6d52482a6a273ffb038f988d589e7bb5cd68e0c8dd8668de'}, 'UserPrivateKey': {1: 'e06f237cdfd516caff0766b5f8ba4e4b2b85824c76910f1374ffce72173f8269'}, 'StoreConfig': {1: '91790b136dba4ddbb5bd8dfcd337a4a8e448cc26b338bf15bf066ba97be56924'}, 'NodeSettingsUpdate': {1: 'ae77fb9f24004635a29abd979f5efa5a75780efe4fec1773cc533ac04aa57482'}, 'NodeSettings': {1: '2735f660f23bfda8ffdf97b8ba3ab1bcdba01461b245832e9a9cb2a661ebcb74'}, 'HTTPConnection': {1: '0a6e181e67978ce45a2414be4c4c39272ca6ed38a5fe9e9877619c13dc6aafef'}, 'PythonConnection': {1: '6ab9e80d2208ce44fb6e5db8427234680f56b1ef8092be701d6d833b6e213926'}, 'DateTime': {1: '34f9942a3f75988a1de6e46c40697698f2505336cf74282f683cfd3a7d6d4ec1'}, 'BlobFile': {1: '229121eb07430f72c66281764a690270ff821a6551c036528b9e749b343bedc1'}, 'SecureFilePathLocation': {1: 'd3e6b95de5da0861922c302e9dabf443ee337b21da695d69c85bdb1e6f0ec45b'}, 'SeaweedSecureFilePathLocation': {1: '0d5c382191c63e68b90237bb4e882abea6311ff1ba645adc784ee272de5f4623'}, 'BlobStorageEntry': {1: 'e010b50076f73fb934029a583310d13c3ec7abaa93520090fae3fb16457868fc'}, 'BlobStorageMetadata': {1: 'f1d0b4085276ba5f15a8cd81553374d465317d96e0a1c427e33e2d866e362d22'}, 'CreateBlobStorageEntry': {1: '183fd3ed16b0687f01979b3b76bbb17a9b029bd39f11d46faf54b3e5205e9e2d'}, 'BlobRetrieval': {1: 'c55f486ea79e9e96c047b32987e555357248cd30f18e1a8244fefe73457e5b9e'}, 'SyftObjectRetrieval': {1: 'da96484c8e57fc060c4dba29ef4e375284720dd05f1ed2ee60e1df52450437cd'}, 'BlobRetrievalByURL': {1: '656a44e91ce560056679b459c9fd33b55a5c2b0754e455099a456074e2e14822'}, 'BlobDeposit': {1: '23a73cc9bff8e6833681e55d872121b6d54520d76f9426fd199a19eb847deea4'}, 'WorkerSettings': {1: '2fe75dd39cb6367bd9cea2c7f59e40a85bbbcfc44f518572f377ef25c3acd205'}, 'HTTPNodeRoute': {1: 'b4662c11f7487ab907caf3cadf8c33eca2e0fbd640ed1fba872c3f982b749986'}, 'PythonNodeRoute': {1: 'd8f268996e5443a248cc2eb5f4a568591e5f9e18952f8519f3f995e1f1f486e4'}, 'EnclaveMetadata': {1: '6ad19306231ebbb4d8b0c4e4cc82b881298835862a2c48f2358db47215a020ac'}, 'DataSubject': {1: '287ab306e1c4ebe0c883600ffd52dc734d08f0217b2a961afbdc6c7883bf4ccd'}, 'DataSubjectCreate': {1: '8b3487af42ba249d7cf705c7f66a09dd832c055814091def34a38133f8148158'}, 'DataSubjectMemberRelationship': {1: '6aed0e1548d6b09bfac132040f7315f49c13855a6bc147a4f1aa4ce09572b952'}, 'Contributor': {1: '3e27f1ea23cecfc3e0956743ae832f0f70ecd144f9df3f128b123e9347944afa'}, 'MarkdownDescription': {1: '506d47fa85728ad444f2fa657e39b341bc759d21a80325831b1e84926ee748f1'}, 'Asset': {1: 'f8370e8dd87df6a05bde1226c52c3ce6e7de636e6032341b977efe5e601a731d'}, 'CreateAsset': {1: 'c3a82856694adcb3c916a961776c2fa3bc90a7ccb50b8b9d42776810266ed241'}, 'Dataset': {1: 'd7a45bf9128472715e63a67192d4181672eadd8b5873d2ba96c2df3b2be749b9'}, 'DatasetPageView': {1: 'c7da1fac28f70c19d199f123b04fbd4a9c7681e3846dee0de70ea926a9440a2e'}, 'CreateDataset': {1: '6a31617de99738bc176f19397884581051443c7c7ba952c983929fed688a9d7e'}, 'ActionDataEmpty': {1: 'fc83d91ac6ba78c749a4e7e16c1aa117baaae62b5f33156ded4e5a79b9532a98'}, 'ActionFileData': {1: '47a0a5f9fb48de09885d4f9a6c5a5a05f4dd04575ea42df7dea0cab02817063f'}, 'Action': {1: '204b3c97b41f208ecb17c541af55e171675faaefa10c103b405a4077b0226a7e'}, 'AnyActionObject': {1: 'f11bd6135ba46247c06bfa8a3c6b7f2e540a1033afe0512c3359e31eb3d59388'}, 'TwinObject': {1: '8f6abd756d41f9639865c6fd55e6cc2ec6b89fd18bac3c77bf3a0502d81ca8ec'}, 'ExactMatch': {1: 'f71a495d2452190596fe435eaf59b07ba390d7895f6c808a2f87a1719227ba9c'}, 'OutputHistory': {1: '24f282dd181ecc7a05c93c02dff84dff45aa52928e2331a15ef667645e9bcf0b'}, 'OutputPolicyExecuteCount': {1: '95d198d2021dbaf9965df39a4571ad06e787684ff79bd6e8a720c47825eebd7e'}, 'OutputPolicyExecuteOnce': {1: 'b8bc1fea2e9b51b5dfc3cbd4b0a131cb2a5b1fe746b5e329395bf319b38bf9b2'}, 'UserPolicy': {1: 'ec3f8ea84e2b05ce56db8f35cff14f78569e921f566477581b3299eb6a9fa053'}, 'SubmitUserPolicy': {1: '3147a4e45270367a40ca8c4b7b502c8683200d123828720365521b90b2775794'}, 'UserCode': {1: 'ad509fccb7bb5c00971453c1f1235785f40d5d5b3eee3df1dc9edafc758c7193'}, 'SubmitUserCode': {1: 'b4a919054beb2488c7b4839d60c494400d791112adf009631dce13787cd58e78'}, 'UserCodeExecutionResult': {1: '9aab24def6616ac908ef1100b654b4dbeca1ea4cfff229c53d6b14491c795555'}, 'CodeHistory': {1: 'bbbd59801717a98691034a37c9de388c7a65db2721bd0c5c9ff0dbe8fc70be67'}, 'CodeHistoryView': {1: '142e78415da10dae739e923d39ce511496a3c7b31e8c4553a6cbb1907c126a3a'}, 'CodeHistoriesDict': {1: '453af101a1de8e37a0bfacf22077c35994718791f295956f1f06727f8d9b7fe8'}, 'UsersCodeHistoriesDict': {1: 'cf8ef92a08cabb068e4616c1700451b75ba4d511343830e3e56b65882fb784aa'}, 'NodePeer': {1: '8920d9e456fd1a13f46c0528a8fe3fec8af46440b3eb89e8d7d48ad64babee1e'}, 'CommandReport': {1: 'a81fe3d0cc5796f45e925d09b6e8132b79fe5df0f341d55b3870c109f8c1e19d'}, 'CommandResult': {1: '14b48d4a1cbc5f5ae1e5e74834e7f1002adae7b2766000ea822f180fd7cd39db'}, 'VPNClientConnection': {1: '7d44711978f930d42c06d04483abcdb1f230782a8f16f4feb5efb7b2b2093bb2'}, 'HeadscaleAuthToken': {1: '0b363503b6c611b44e33561a2a6db3f260cfd4bbc5f4245deac5052fd5149803'}, 'TailscalePeer': {1: '8ff85aa2b913a6bb206b9de0495d9f74a17f55823891da98cb6fdbe78f46a44b'}, 'TailscaleStatus': {1: 'ed262f4b9a569d9933f4a86cd2caa2ce213fc7a2319a1371f6a3cf3ccf884c8a'}, 'OnDiskBlobDeposit': {1: 'da3abda453def0d7c70c8a5dfcc3c8d00dd6822f60ddc01be3bdead4b0b5b482'}, 'SeaweedFSBlobDeposit': {1: 'bcbec5dcdc06a0c87f89a10a6a8809706f24cedd97b5f850f8b48840a1f41941'}, 'NumpyArrayObject': {1: 'd47a376401d92d47e5748e34f98ee270f8ebfd52cffbe6271b5faa8193e728c5'}, 'NumpyScalarObject': {1: '952bebb4dd3e3641c33b4ebcf2c051dbdebae5f1bf3b7b63ea89423360705411'}, 'NumpyBoolObject': {1: 'b7a231baaa4b1f519d70c5afb15b4a9b7232f1128f7fd3709c1ea8b7345f8c6c'}, 'PandasDataframeObject': {1: 'ff9d6c1884413f712d95d29190e30938b33de19e11dff9f88d9b89c51499cac5'}, 'PandasSeriesObject': {1: '69eadfe14e5a7035767d2538e2db8775da6569cf5127f58d13315c4b85e5603d'}, 'ReplyNotification': {1: 'ce1e2a6b0d618478d3b1b992e4c8605817919c88a4884ca0540e0886ecdb8215'}, 'Notification': {1: '1e5a65d91e27bf53d5b2ed0b45d9cee0cf77104b7111f99223194ceb0d0137fe'}, 'CreateNotification': {1: '6858b743ac07d853a0302dc64de0e7d852135e2564ebad325d5ff35d17c29f6f'}, 'Change': {1: '2c470ff8aa076b88105640ce79d361a9b439927e501c238fa33ac7c1c45aa2c0'}, 'ChangeStatus': {1: '7571229e92d8d52a0e90fa8856169b41045b42b50568b266823bdcea838dfb39'}, 'ActionStoreChange': {1: 'cf527995930cce09d90806713d30301493240079319bcc85e894428aee46017e'}, 'Request': {1: '340f4ac61ccbf6f566666327d6bca043dcd643e6f8e24897ef10bd6312e74995'}, 'RequestInfo': {1: 'd571708de3c187ca5840c64784d99f7bfce8f33aa2ba48f9d56b824564551654'}, 'RequestInfoFilter': {1: 'c336af8d474071eb61e5f467513753e64d4e153e12892f9c6875b235057b0f0a'}, 'SubmitRequest': {1: '1870ce541169eab04cb69d3ae88ea30dc2fcdd997b620567ca9d87936d9600cf'}, 'ObjectMutation': {1: '275d9cf180904d1e34e1f3d7e838105e843127faf5a64029a1cf85d00234b8c9'}, 'EnumMutation': {1: '3a1d1b47e0cdb5094298bce58bc9b76ecb66064459504f910c7d755eb1d5e276'}, 'UserCodeStatusChange': {1: '928dd4ceeb4858b18b806ca62b49a840f54269f7866744c9aa6edb0af9d7dfc1'}, 'SyftObjectMigrationState': {1: '194fd4dc57764d454ac763d256e3bfcd2b0040a134daf9ee0d8e5ac7ab21abbc'}, 'ProjectThreadMessage': {1: '9f8b11d603caae6d0e0f28957949dfc57c26fec9685f2c80501330b1d6bae665'}, 'ProjectMessage': {1: 'd678beafc33f7e7df7e771a82d5cba6d5a36728a033d3976b6e5998726733d27'}, 'ProjectRequestResponse': {1: '51b5a5d8cf0bde45abd2bd3a4411f93769fa542666a137ce9611d38fb48ffb4c'}, 'ProjectRequest': {1: '9eff1b3cc74c9706722363abb4062fc77c0a4f093d448b795ad662861649f111'}, 'AnswerProjectPoll': {1: 'f538a4fcae286cbc9755f51e2f2ce8809d66ce5d66f50173ef1824f89ce9b51d'}, 'ProjectPoll': {1: 'b456a699a249fd3fffe9739cdd9ec3ee8c05e59b2d9872ad9864167d78088091'}, 'Project': {1: 'bf59890e92d95b362cc7ef9c3d7fa6a1815978e02111a30cbcb047239e57d61e'}, 'ProjectSubmit': {1: '5084844056ddefcea7fc634dd9945c03ef6d030bcd8f63aa07fe11fea0a5389f'}, 'QueueItem': {1: '1d53446d5cd788120b15ea6b108a4a7abd480377370be7128f44297f8fb00b76'}, 'ZMQClientConfig': {1: 'e3153f18c9fd04cf07b844153d093c8a090baac4c99d71ecd6491961e7f1dafb'}, 'Plan': {1: '41713fc89a2cab7db592df6cd1c45e1309f86a50a8f531ddaf4052947186b0e0'}}

In [ ]:
import json

In [ ]:
json.dumps(x)

In [ ]:

# Exception: BaseConfig <class 'syft.service.service.ServiceConfig'> version 1 hash has changed.
# 332964663526a4d6ddc64bc9d77aa9324000bb721520a5465229235523fe8c7d not in 
# dict_values([{'version': 1, 'hash': 'f35db90c1c3bdd993b7b1e4f28327a8f6b1d0e43115b044167ca28d5c740178a', 'action': 'add'}]). You probably need to bump the version number.


In [ ]:
# k syft.service.service.ServiceConfig BaseConfig <class 'syft.service.service.ServiceConfig'> 1 332964663526a4d6ddc64bc9d77aa9324000bb721520a5465229235523fe8c7d
# got versions for canonical name BaseConfig {1: {'version': 1, 'hash': 'f35db90c1c3bdd993b7b1e4f28327a8f6b1d0e43115b044167ca28d5c740178a', 'action': 'add'}}
# is versoin in versions keys 1 dict_keys([1]) True
# is hash in hash values 332964663526a4d6ddc64bc9d77aa9324000bb721520a5465229235523fe8c7d dict_values([{'version': 1, 'hash': 'f35db90c1c3bdd993b7b1e4f28327a8f6b1d0e43115b044167ca28d5c740178a', 'action': 'add'}]) False
# else 1 dict_keys([1]) True

In [ ]:
import json

In [ ]:
import re

def natural_key(key: int | str) -> list[int]:
    """Define key for natural ordering of strings."""
    if isinstance(key, int):
        key = str(key)
    return [int(s) if s.isdigit() else s for s in re.split('(\d+)', key)]

def sort_dict_naturally(d: dict) -> dict:
    """Sort dictionary by keys in natural order."""
    return {k: d[k] for k in sorted(d.keys(), key=natural_key)}


# Example usage:
d = {
    "apple10": "value10",
    "apple2": "value2",
    "banana": "value_banana",
    "apple1": "value1",
}

sorted_d = sort_dict_naturally(d)
print(sorted_d)


In [ ]:
from syft.types.syft_object import SyftMigrationRegistry, SyftObjectRegistry
from syft.service.metadata.migrations import *

In [ ]:
from typing import Optional

In [ ]:
from collections import defaultdict
def protocol_state_builder(protocol_dict: dict, stop_key: Optional[str] = None):
    sorted_dict = sort_dict_naturally(protocol_dict)
    state_dict = defaultdict(dict)
    for k, v in sorted_dict.items():
        # stop early
        if stop_key == k:
            return state_dict
        print("k", k, v)
        object_versions = sorted_dict[k]["object_versions"]
        print("got object versions", object_versions)
        for canonical_name, object_metadata in object_versions.items():
            action = object_metadata["action"]
            version = object_metadata["version"]
            hash_str = object_metadata["hash"]
            print("canonical", canonical_name, object_metadata)
            versions = state_dict[canonical_name]
            if action == "add" and (version in versions.keys() or hash_str in versions.values()):
                raise Exception(f"Can't add {object_metadata} already in state {versions}")
            elif action == "remove" and (version not in versions.keys() or hash_str not in versions.values()):
                raise Exception(f"Can't remove {object_metadata} missing from state {versions}")
            if action == "add":
                versions[version] = hash_str
            elif action == "remove":
                del versions[version]
            state_dict[canonical_name] = versions
    return state_dict

In [ ]:
a = {}
a["1"] = {}
a["1"]["object_versions"] = {}
a["1"]["object_versions"]["Thing"] = {}
a["1"]["object_versions"]["Thing"]["version"] = 1
a["1"]["object_versions"]["Thing"]["hash"] = "abc"
a["1"]["object_versions"]["Thing"]["action"] = "add"

a["2"] = {}
a["2"]["object_versions"] = {}
a["2"]["object_versions"]["Thing"] = {}
a["2"]["object_versions"]["Thing"]["version"] = 2
a["2"]["object_versions"]["Thing"]["hash"] = "def"
a["2"]["object_versions"]["Thing"]["action"] = "add"

a["3"] = {}
a["3"]["object_versions"] = {}
a["3"]["object_versions"]["Thing"] = {}
a["3"]["object_versions"]["Thing"]["version"] = 1
a["3"]["object_versions"]["Thing"]["hash"] = "abc"
a["3"]["object_versions"]["Thing"]["action"] = "remove"

In [ ]:
a

In [ ]:
b = protocol_state_builder(a)

In [ ]:
b

In [ ]:
sy.serde.recursive.TYPE_BANK["numpy.ndarray"]

In [ ]:
# (
#         nonrecursive,
#         serialize,
#         deserialize,
#         attribute_list,
#         exclude_attrs_list,
#         serde_overrides,
#         hash_exclude_attrs,
#         cls,
#         attribute_types,
#         version,
#     ) = sy.serde.recursive.TYPE_BANK["syft.service.metadata.node_metadata.NodeMetadataV2"]

In [ ]:
# sy.serde.recursive.TYPE_BANK.keys()

In [ ]:
def _calculate_object_hash(klass) -> str:
    # TODO: this depends on what is marked as serde
    field_data = {
        field_name: repr(model_field.annotation)
        for field_name, model_field in klass.__fields__.items()
    }
    obj_meta_info = {
        "canonical_name": klass.__canonical_name__,
        "version": klass.__version__,
        "unique_keys": getattr(klass, "__attr_unique__", []),
        "field_data": field_data,
    }

    return hashlib.sha256(json.dumps(obj_meta_info).encode()).hexdigest()

In [ ]:
from syft.types.syft_object import SyftBaseObject
import hashlib

In [ ]:
def diff_state(state: dict) -> dict:
    object_diff = {}
    compare_dict = {}
    type_bank = sy.serde.recursive.TYPE_BANK
    for k in type_bank:
        (
            nonrecursive,
            serialize,
            deserialize,
            attribute_list,
            exclude_attrs_list,
            serde_overrides,
            hash_exclude_attrs,
            cls,
            attribute_types,
            version,
        ) = type_bank[k]
        if issubclass(cls, SyftBaseObject):
            hash_str = _calculate_object_hash(cls)
            canonical_name = cls.__canonical_name__
            print("k", k, canonical_name, cls, version, hash_str)

            # build this up for later
            compare_dict[canonical_name] = {}
            compare_dict[canonical_name][version] = hash_str

            if canonical_name not in state:
                # new object so its an add
                object_diff[canonical_name] = {}
                object_diff[canonical_name]["version"] = version
                object_diff[canonical_name]["hash"] = hash_str
                object_diff[canonical_name]["action"] = "add"
                continue
            
            versions = object_diff[canonical_name]

            if version in versions.keys() and hash_str in versions.values():
                # already there so do nothing
                continue
            elif version in versions.keys():
                raise Exception(f"{canonical_name} {cls} version {version} hash has changed. You probably need to bump the version number.")
            else:
                # new object so its an add
                object_diff[canonical_name] = {}
                object_diff[canonical_name]["version"] = version
                object_diff[canonical_name]["hash"] = hash_str
                object_diff[canonical_name]["action"] = "add"
                continue

    # now check for remove actions
    for canonical_name in state:
        for version, hash_str in state[canonical_name].items():
            if canonical_name not in compare_dict:
                # missing so its a remove
                object_diff[canonical_name] = {}
                object_diff[canonical_name]["version"] = version
                object_diff[canonical_name]["hash"] = hash_str
                object_diff[canonical_name]["action"] = "remove"
                continue
            versions = compare_dict[canonical_name]
            if version in versions.keys():
                # missing so its a remove
                object_diff[canonical_name] = {}
                object_diff[canonical_name]["version"] = version
                object_diff[canonical_name]["hash"] = hash_str
                object_diff[canonical_name]["action"] = "remove"
                continue
    return object_diff

In [ ]:
g = diff_state(b)

In [ ]:
g

In [ ]:
a["1"] = 1

In [ ]:
a["2"] = 2

In [ ]:
a["11"] = 1

In [ ]:
a["dev"] = 4

In [ ]:
a

In [ ]:
y = json.dumps(a)

In [ ]:
y

In [ ]:
d = json.loads(y)
d

In [ ]:
d

In [ ]:
for k,v in d.items():
    print("k", k, v)

In [ ]:
a[1] = "a"

In [ ]:
sorted_d = sort_dict_naturally(a)

In [ ]:
sorted_d

In [ ]:
object_versions = dict()
object_versions["SyftObject"] = {"version": "1", "hash": "25a574002025025cfd155e3970305293e21fdd6af9dcde176990802306cc0359", "action":"add"}

In [ ]:
a["1"] = object_versions

In [ ]:
a